In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pylab
from matplotlib.mlab import griddata
import xgboost as xgb
%matplotlib inline

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import graphviz
from sklearn import preprocessing
#from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import KFold

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
#from sklearn.model_selection import GridSearchCV

In [4]:
kansas_data = pd.read_csv('kansas_data_cleaned.csv')

/Users/apple/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
kansas_data.head()

,Unnamed: 0,id,agency,daterept,hourrept,ucrcode,offense,neighborhood,subdivision,spx,...,Minute,Time,Date,Day,DayOfWeek,Week,Month,Year,FullDayOfWeek,FullMonth
0,1,2,JCSO,7/3/15 0:00,15:14:00,35A,POSSESSION OF HALLUCINOGENIC DRUG,NaN,-1.0,2144629.87,...,14,914,2015-07-03,3,4,27,7,2015,Friday,Jul
1,2,3,JCSO,6/27/05 0:00,13:46:00,90Z,DRIVING WHILE HABITUAL VIOLATOR,NaN,-1.0,2149483.00,...,46,826,2005-06-27,27,0,26,6,2005,Monday,Jun
2,3,4,JCSO,2/5/05 0:00,01:45:00,90Z,FAILURE TO APPEAR,NaN,-1.0,2149483.00,...,45,105,2005-02-05,5,5,5,2,2005,Saturday,Feb
3,4,5,JCSO,4/14/10 0:00,19:52:00,90D,DRIVING UNDER INFLUENCE OF ALCOHOL OR DRUGS; U...,JC1,10200.0,2149561.34,...,52,1192,2010-04-14,14,2,15,4,2010,Wednesday,Apr
4,5,6,JCSO,3/18/11 0:00,21:15:00,90Z,CONTEMPT OF COURT; INDIRECT,JC1,10200.0,2149561.34,...,15,1275,2011-03-18,18,4,11,3,2011,Friday,Mar


In [6]:
kansas_data=kansas_data.drop(kansas_data.columns[0],axis=1)

In [7]:
#kansas_data.head()

In [8]:
kansas_data.shape

(335484, 35)

In [9]:
kansas_data.Year.value_counts()

2010    27266
2006    27026
2007    26696
2015    26602
2012    26458
2011    26320
2004    26268
2009    26100
2008    26089
2013    25110
2014    24818
2005    24415
2003    19851
2002     2165
2001       79
2000       70
1999       56
1998       45
1997        7
1993        5
1995        4
1991        4
1990        3
1982        3
1989        3
1996        3
1992        2
1986        2
1984        2
1983        2
1980        2
1988        1
1987        1
1994        1
1981        1
1979        1
1978        1
1977        1
1975        1
Name: Year, dtype: int64

### In the following analysis, we only consider the crime data of recent five years.

In [10]:
data_5 = kansas_data[2010 < kansas_data['Year']]

In [11]:
data_5.Year.value_counts()

2015    26602
2012    26458
2011    26320
2013    25110
2014    24818
Name: Year, dtype: int64

In [12]:
data_5.ucrcode.value_counts()

23     33287
90Z    15831
13B    13996
290     9756
35A     8701
90D     8013
220     5592
13C     4016
23H     3698
240     3529
26C     2863
90C     2410
250     2120
13A     2023
26B     1738
35B     1685
26A     1603
90G     1214
90J     1159
11D      864
11A      677
280      655
100      527
120      499
90A      444
520      432
370      411
200      269
40A      264
40B      261
90F      191
270      167
11B      166
36B       71
90E       60
09A       40
210       34
90I       18
39A       10
09B        5
36A        4
510        2
09C        1
39B        1
26D        1
Name: ucrcode, dtype: int64

In [13]:
data_5.ucrcode.value_counts()[:-12].index

Index([u'23', u'90Z', u'13B', u'290', u'35A', u'90D', u'220', u'13C', u'23H',
       u'240', u'26C', u'90C', u'250', u'13A', u'26B', u'35B', u'26A', u'90G',
       u'90J', u'11D', u'11A', u'280', u'100', u'120', u'90A', u'520', u'370',
       u'200', u'40A', u'40B', u'90F', u'270', u'11B'],
      dtype='object')

In [14]:
#if '26D' in data_5.ucrcode.value_counts()[:-12].index:
   # print 1

In [15]:
data_5.premisedescription.value_counts()

Single Residence (includes attached garage)           26327
City Street                                           14390
Government / Public Building                          12122
Apartment Complex                                     11142
Parking Lot/ or Parking Garage                         9261
Department / Discount Store                            7429
Duplex/ Individual multiple housing units              4209
Public / Private Primary or Secondary School           3670
Specialty Store                                        3287
Highway                                                2966
Commercial / Office Building                           2312
Grocery / Supermarket                                  2002
Restaurant                                             2000
Other                                                  1933
Hotel / Motel / Inn                                    1803
Service / Gas Station                                  1128
Jail / Prison                           

In [16]:
data_5 = data_5.rename(columns = {'premisedescription': 'location_type'})

In [17]:
#data_5.location_type.value_counts().index

In [18]:
len(data_5.location_type.value_counts().index)

76

In [19]:
def missing_counts(dataset):
    missing ={}
    for col in dataset.columns:
        missing[col] = len(dataset[dataset[col].isnull()])
    return missing

In [20]:
missing_counts(data_5)

{'Date': 0,
 'Day': 0,
 'DayOfWeek': 0,
 'FullDayOfWeek': 0,
 'FullMonth': 0,
 'Hour': 0,
 'Minute': 0,
 'Month': 0,
 'Time': 0,
 'Week': 0,
 'Year': 0,
 'agency': 1,
 'apid': 35579,
 'city': 0,
 'crimetype': 0,
 'daterept': 0,
 'fulllocation': 0,
 'geom': 0,
 'hourrept': 0,
 'id': 0,
 'inactivitydescription': 12706,
 'incidentid': 0,
 'location': 0,
 'location_type': 11528,
 'neighborhood': 28820,
 'offense': 0,
 'spx': 0,
 'spy': 0,
 'state': 772,
 'subdivision': 12520,
 'theftdescription': 92262,
 'ucrcode': 0,
 'x': 0,
 'y': 0,
 'zip': 1959}

In [21]:
data_5.location.value_counts()

100 BLOCK N KANSAS AVE                 2506
7700 BLOCK MISSION RD                  2384
27700 BLOCK W 159TH ST                 1812
4800 BLOCK TOWN CENTER DR              1060
11800 BLOCK W 95TH ST                  1038
13600 BLOCK S ALDEN ST                  722
500 BLOCK E OLD 56 HWY                  718
12400 BLOCK FOSTER ST                   713
15300 BLOCK W 119TH ST                  594
11200 BLOCK W 95TH ST                   558
700 BLOCK N 7TH ST TFWY                 536
7700 BLOCK FRONTAGE RD                  521
300 BLOCK N K7 HWY                      501
11500 BLOCK METCALF AVE                 373
5800 BLOCK ANTIOCH RD                   372
12500 BLOCK W 87TH ST PKWY              354
100 BLOCK N CHERRY ST                   330
600 BLOCK E PRAIRIE ST                  323
11500 BLOCK W 95TH ST                   309
15700 BLOCK METCALF AVE                 300
12200 BLOCK S STRANG LINE RD            298
11100 BLOCK W 95TH ST                   298
11700 BLOCK METCALF AVE         

In [22]:
len(data_5.location.value_counts().index)

22384

In [23]:
data_5['ucrcode'] = data_5['ucrcode'].astype('category')

In [24]:
data_5['Category'] = data_5.ucrcode.apply(lambda x: x if x in data_5.ucrcode.value_counts()[:-12].index else 'others')

In [25]:
#data_5.Category.value_counts()

### Now we have aggregated categories with small number of crimes to a new category called 'others'.

## Next we want to build features for our model.

In [26]:
data_5 = data_5.reset_index(drop = True)

In [27]:
data_5['Day'] = data_5['Day'].astype('category')
data_5['DayOfWeek'] = data_5['DayOfWeek'].astype('category')
data_5['Hour'] = data_5['Hour'].astype('category')
data_5['Month'] = data_5['Month'].astype('category')
data_5['Week'] = data_5['Week'].astype('category')
data_5['Year'] = data_5['Year'].astype('category')

In [35]:
location_columns = ['city', 'location']

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
le = LabelEncoder()
for feature in location_columns:
    data_5[feature] = le.fit_transform(data_5[feature])

In [38]:
#len(set(data_5['city']))

In [39]:
label = data_5['Category'].astype('category')

In [40]:
data_5['city'] = data_5['city'].astype('category')
data_5['location'] = data_5['location'].astype('category')

In [41]:
#label.to_csv('label_data_5.csv')

In [42]:
features = ['Day',          # indicates which day in a month
            'DayOfWeek',    # indicates which day in a week
            'Hour',         # indicates which hour in a day
            'Month',        # indicates which month in a year
            'Week',         # indicates which week in a year
            'Year',         # indicates which year in recent five years
            'city',         #indicates which city in Kansas State
            #'location',     #indicates which street in the city
            'Time',         #indicates the time period in terms of minutes in a day
            'x',            # indicates the x coordinates of the location
            'y'             # indicates the y coordinates of the location
           ]

In [43]:
new_data = data_5[features]

In [44]:
def onehot_transform(X):
    dummies_X = pd.get_dummies(X)
    return dummies_X

In [45]:
new_data = onehot_transform(new_data)

In [46]:
new_data.head()

,Time,x,y,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,...,city_34,city_35,city_36,city_37,city_38,city_39,city_40,city_41,city_42,city_43
0,914,-95.073315,38.941743,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1275,-95.055912,38.943095,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1398,-95.055912,38.943095,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,219,-95.055912,38.943095,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,165,-95.055912,38.943095,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
new_data.shape

(129308, 178)

In [51]:
#new_data.to_csv('feature_data_5.csv')

## Now we've finished building up features.

In [52]:
feature_list = new_data.columns

In [53]:
feature_list

Index([u'Time', u'x', u'y', u'Day_1', u'Day_2', u'Day_3', u'Day_4', u'Day_5',
       u'Day_6', u'Day_7',
       ...
       u'city_34', u'city_35', u'city_36', u'city_37', u'city_38', u'city_39',
       u'city_40', u'city_41', u'city_42', u'city_43'],
      dtype='object', length=178)

## Fitting and Tuning an algorithm. We start with a simple example.

### We split the data into training set (80%) and test set (20%) using stratified splits (preserving the percentage of samples for each class)

In [54]:
from sklearn.cross_validation import StratifiedShuffleSplit

In [55]:
strat_shuffle = StratifiedShuffleSplit(label, n_iter = 1, test_size = 0.2, random_state = 42)

In [56]:
strat_shuffle

StratifiedShuffleSplit(labels=['35A' '90Z' '90D' ..., '26B' '290' '26C'], n_iter=1, test_size=0.2, random_state=42)

In [57]:
for train_index, test_index in strat_shuffle:
    X_train, X_test = new_data.iloc[train_index], new_data.iloc[test_index]
    y_train, y_test = label.iloc[train_index], label.iloc[test_index]

## Next we want to fit an XGBoost classifier to our model.

### First we do the normalization of X_train and X_test.

In [58]:
from sklearn.preprocessing import StandardScaler

In [60]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [61]:
from xgboost import XGBClassifier

In [62]:
num_class = len(set(label))

### We start with one particular parameter set.

In [63]:
param_dist ={'objective': 'multi:softmax',
             'learning_rate': 0.4,
             'silent': 0,
             'nthread': 4,
             'max_depth': 8,
             'min_child_weight': 1,
             'gamma': 0,
             'reg_alpha': 0.05,
             'subsample': 0.8,
             'colsample_bytree': 0.8,
             'max_delta_step': 1,
             'scale_pos_weight':1    
}

In [64]:
clf = xgb.XGBClassifier(**param_dist)

In [65]:
clf.fit(X_train, y_train, eval_metric = 'merror')

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.4, max_delta_step=1, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='multi:softprob', reg_alpha=0.05, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=0, subsample=0.8)

In [66]:
predictions = clf.predict(X_test)

In [67]:
predictions

array(['290', '23', '90Z', ..., '90Z', '290', '240'], dtype=object)

In [69]:
from sklearn import metrics

In [70]:
from sklearn.metrics import confusion_matrix

In [71]:
from sklearn.metrics import classification_report

In [72]:
confusion_matrix(y_test, predictions)

array([[   0,    0,    0, ...,    0,    1,    0],
       [   0,    0,    0, ...,    0,    4,    0],
       [   0,    0,    0, ...,    0,    3,    0],
       ..., 
       [   0,    0,    0, ...,    2,    5,    0],
       [   1,    0,    0, ...,    0, 2142,    1],
       [   0,    0,    0, ...,    0,    1,    0]])

In [95]:
#confusion_matrix(y_test, predictions).shape

In [73]:
#len(set(label))

In [74]:
list(set(label))

['35B',
 '35A',
 '90Z',
 '270',
 '90C',
 '90A',
 '90F',
 '90G',
 '90D',
 '90J',
 '250',
 '26C',
 '26B',
 '26A',
 '23',
 '520',
 '280',
 '200',
 '40B',
 '40A',
 '11D',
 '120',
 '240',
 'others',
 '23H',
 '100',
 '220',
 '13C',
 '13B',
 '13A',
 '11A',
 '11B',
 '370',
 '290']

In [75]:
target_names = list(set(label))

In [76]:
classification_report(y_test, predictions, target_names = target_names)

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'             precision    recall  f1-score   support\n\n        35B       0.00      0.00      0.00       105\n        35A       0.00      0.00      0.00       135\n        90Z       0.00      0.00      0.00        33\n        270       0.22      0.01      0.02       173\n        90C       0.00      0.00      0.00       100\n        90A       0.15      0.01      0.02       405\n        90F       0.27      0.30      0.28      2799\n        90G       0.12      0.01      0.03       803\n        90D       0.33      0.02      0.04        54\n        90J       0.25      0.05      0.08      1118\n        250       0.42      0.85      0.56      6658\n        26C       0.50      0.81      0.61       740\n        26B       0.18      0.04      0.07       706\n        26A       0.46      0.08      0.14       424\n         23       0.14      0.01      0.02       321\n        520       0.47      0.04      0.08       348\n        280       0.34      0.14      0.20       573\n        200       1.00   

In [77]:
u, index = np.unique(predictions, return_index=True)
u

array(['100', '11A', '11B', '11D', '120', '13A', '13B', '13C', '200',
       '220', '23', '23H', '240', '250', '26A', '26B', '26C', '270', '290',
       '35A', '35B', '370', '40A', '40B', '520', '90A', '90C', '90D',
       '90F', '90G', '90J', '90Z', 'others'], dtype=object)

In [78]:
len(u)

33

In [79]:
from sklearn.metrics import f1_score

In [80]:
f1_score(y_test, predictions, average = 'weighted')

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.36448996311442061

In [81]:
f1_score(y_test, predictions, average = 'macro')

0.15977871770831117

In [82]:
f1_score(y_test, predictions, average = 'micro')

0.43357048952130539

In [83]:
from sklearn.metrics import accuracy_score

In [84]:
accuracy_score(y_test, predictions)

0.43357048952130539

In [96]:
f1_score(y_test, predictions, average = 'weighted')

0.36448996311442061

### The above shows the result of prediction using a single parameter set of a XGboost classifier.

In [86]:
### Let's try to use the same parameter set but using multi: softprob and eval_metric = 'mlogloss' to see what happens.

In [87]:
#param_dist_2 ={'objective': 'multi:softprob',
    #         'learning_rate': 0.4,
    #         'silent': 0,
     #        'nthread': 4,
      #       'max_depth': 8,
       #      'min_child_weight': 1,
       #      'gamma': 0,
       #      'reg_alpha': 0.05,
        #     'subsample': 0.8,
        #     'colsample_bytree': 0.8,
        #     'max_delta_step': 1,
        #     'scale_pos_weight':1    
#}

In [88]:
#clf = xgb.XGBClassifier(**param_dist_2)

In [89]:
#clf.fit(X_train, y_train, eval_metric = 'mlogloss')

In [90]:
#label.shape[0]

In [91]:
#predictions_prob = clf.predict(X_test)

In [92]:
#predictions_prob 

In [93]:
#sum(predictions != predictions_prob)

In [94]:
### It's the same, using multi:softmax or multi: softprob